Import semua library yang dibutuhkan, input citra lalu ubah dalam bentuk grayscale, disertai dengan pengambilan value pixel dari citra

In [1]:
from PIL import Image
import numpy as np
from collections import Counter
import heapq

img_path = 'asset/pisang4_grayscale.jpg'
img = Image.open(img_path)

img_gray = img.convert("L")

pixels = np.array(img_gray).flatten()

Mendefinisikan Huffman Tree Node, dilakukan perhitungan pixel sebagai frekuensi, dan pembuatan class Node

In [2]:
freq = Counter(pixels)

# Setiap node mempunyai left, right, dan root. fungsi akan membandingkan node left dan right berdasarkan rootnya.
class Node:
    def __init__(self, left=None, right=None, root=None):
        self.left = left
        self.right = right
        self.root = root

    def children(self):
        return (self.left, self.right)

    def __lt__(self, other):
        return self.root < other.root

Pembuatan fungsi huffman tree, heap akan diinisialisasi dengan nodes pada setiap pixel dan frekuensinya. Lalu fungsi akan menggabungkan kedua nodes dengan frekuensi terendah hingga satu node yang tersisa, node terakhir ini menggambarkan root dari huffman treenya.

In [3]:
def build_huffman_tree(frequencies):
    heap = [Node(root=freq, left=symbol) for symbol, freq in frequencies.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        node = Node(left=left, right=right, root=left.root + right.root)
        heapq.heappush(heap, node)
    return heap[0]

Pembuatan fungsi assign codes, fungsi akan bekerja secara rekursif untuk memasukkan kode binary ke dalam tiap tiap leaf node pada huffman tree.

In [4]:
def assign_codes(node, prefix="", code={}):
    if isinstance(node.left, Node):
        assign_codes(node.left, prefix + "0", code)
    else:
        code[node.left] = prefix or "0"
    if isinstance(node.right, Node):
        assign_codes(node.right, prefix + "1", code)
    else:
        code[node.right] = prefix or "0"
    return code

Penggunaan fungsi huffman tree dan assign code, disertai dengan sedikit list dari huffman tree yang telah di assign tadi.

In [5]:
huffman_tree = build_huffman_tree(freq)
huffman_codes = assign_codes(huffman_tree)

list(huffman_codes.items())[:10]

[(70, '0000000'),
 (None, '1111111'),
 (139, '0000001'),
 (107, '000001'),
 (106, '000010'),
 (129, '000011'),
 (85, '0001000'),
 (197, '00010010'),
 (62, '0001001100'),
 (47, '00010011010')]

setelah dilakukan fungsi build tree dan assign code, citra asli di encode dengan mengganti tiap pixelnya dengan pixel baru hasil huffman code. dilakukan juga kalkulasi original size, compressed size, serta perbandingan keduanya.

In [6]:
encoded_image = ''.join(huffman_codes[pixel] for pixel in pixels)

original_bits = len(pixels) * 8

encoded_bits = len(encoded_image)

compression_ratio = original_bits / encoded_bits

original_bits, encoded_bits, compression_ratio

(1384448, 1233440, 1.1224283305227656)

fungsi decode huffman akan mengubah string hasil encode tadi ke dalam bentuk nilai" pixel menggunakan huffman code. Nilai pixel hasil decode digunakan untuk membuat citra baru yang terkompress. 

In [7]:
def decode_huffman(encoded_str, huffman_codes):
    reverse_huffman_codes = {v: k for k, v in huffman_codes.items()}
    current_code = ''
    decoded_pixels = []
    for bit in encoded_str:
        current_code += bit
        if current_code in reverse_huffman_codes:
            decoded_pixels.append(reverse_huffman_codes[current_code])
            current_code = ''
    return decoded_pixels

decoded_pixels = decode_huffman(encoded_image, huffman_codes)

decoded_image = Image.new('L', img_gray.size)
decoded_image.putdata(decoded_pixels)
decoded_image_path = 'asset/pisang4_compress.jpg'
decoded_image.save(decoded_image_path)

decoded_image_path


'asset/pisang4_compress.jpg'